In [1]:
import hexathello.aiPlayers as aiPlayers
import hexathello.AutoPlayer as autoPlayer
import hexathello.Engine as engine
import hexathello.history as history
import hexathello.jable as jable
import hexathello.printing as printing

import numpy as np

from os import path
import os
import math
import tensorflow as tf

from os import listdir
from os.path import join, isfile


# -- Settings
game_size: int = 5
player_count: int = 2

In [3]:
matchup_dir = "data/history/matchups_layers"

# update with new JSON file
matchup_files = [f for f in listdir(matchup_dir) if f.endswith("manydata.json")]
print(matchup_files)

agent_ids = [
    "kha_layers_1",
    "kha_layers_2",
    "kha_layers_3",
    "kha_layers_4",
    "kha_layers_5",
    "kha_layers_6",
    "kha_layers_7",
    "kha_layers_8",
    "kha_layers_9"
]

['all_matchups_manydata.json']


In [4]:
for ai_id in agent_ids:
    print(f"\n Retraining agent: {ai_id}")

    training_pov = []

    relevant_files = [f for f in matchup_files if ai_id in f]

    match_frames = []
    for fname in matchup_files:
        full_path = join(matchup_dir, fname)
        
        history_fromDisk: jable.JyFrame = jable.read_file(
            full_path
        )
    
        history_decoded: jable.JyFrame = history.history_fromInt(
            history_fromDisk
        )
    
        if len(history_decoded) < 20:
            print('skipping')
            continue
    
    
        povHistory: jable.JyFrame = history.povHistory_from_literalHistory(
            history_decoded
        )

        del history_decoded
        del history_fromDisk

        match_frames.append(povHistory)


        input_size: int = len(povHistory[0, 'board_state'])
        output_size: int = len(povHistory[0, 'player_action'])

    keras_path = join("data", "ai", "layer_sweeper", f"{ai_id}.keras")
    if not path.isfile(keras_path):
        print(f"Model not found at {keras_path}")
        continue

    brain_model = tf.keras.models.load_model(keras_path)
    os.makedirs(os.path.dirname(keras_path), exist_ok=True)

    ai_agent = aiPlayers.KerasHexAgent(
        size=game_size,
        player_count=player_count,
        brain=brain_model,
        player_id=None,
        ai_id=ai_id
    )

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=keras_path,
        monitor='loss',
        mode='min',
        save_best_only=True
    )

    training_pov_frame: jable.JyFrame = jable.JyFrame(training_pov)

    for povHistory in match_frames:
        ai_agent.train(
            game_history=povHistory,
            epochs=10,
            callbacks=[checkpoint_cb]
        )

    print(f"Retrained and saved model as {keras_path}")



 Retraining agent: kha_layers_1
Epoch 1/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.0267
Epoch 2/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.8584
Epoch 3/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.8129
Epoch 4/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.7843
Epoch 5/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.7657
Epoch 6/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.7412
Epoch 7/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.7330
Epoch 8/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.7193
Epoch 9/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.6959
Epoch 10/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.6875
Retrained and saved model as data\ai\layer_sweeper\kha_layers_1.keras

 Retraining agent: kha_layers_2
Epoch 1/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.1145
Epoch 2/10
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8857
Epoch 3/10
1662/1662 ━━━━━━━━━━━━━━